# 🔍 АНАЛИЗ КАЧЕСТВА ВАШЕГО MZA

## 🎯 ЦЕЛЬ
Детальный анализ качества определения рыночных зон вашим MZA на разных таймфреймах

## 📊 МЕТРИКИ ОЦЕНКИ
1. **Economic Value (40%)** - разделение доходности по зонам
2. **Zone Stability (25%)** - стабильность зон  
3. **Signal Consistency (20%)** - точность сигналов
4. **Adaptability Score (15%)** - разумность параметров


In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Библиотеки импортированы")


✅ Библиотеки импортированы


In [2]:
# Импорт MZA модулей
try:
    from data_loader import load_btc_data
    from accurate_mza_classifier import AccurateMZAClassifier
    print("✅ MZA модули импортированы")
except ImportError as e:
    print(f"❌ Ошибка импорта: {e}")
    print("💡 Убедитесь, что файлы находятся в правильной директории")


✅ MZA модули импортированы


In [3]:
# Класс для анализа качества MZA
class MZAQualityAnalyzer:
    """Анализатор качества определения рыночных зон MZA"""
    
    def __init__(self):
        self.results = {}
        
    def analyze_mza_quality(self, data: pd.DataFrame, zones: np.ndarray, 
                           params: dict, timeframe: str = "15m") -> dict:
        """Полный анализ качества MZA"""
        print(f"🔍 АНАЛИЗ КАЧЕСТВА MZA ДЛЯ {timeframe}")
        print("=" * 50)
        
        # Вычисляем доходность
        returns = data['close'].pct_change().dropna()
        
        # Выравниваем индексы
        max_lookback = max(params.get('slowMALength', 50), params.get('adxLength', 14))
        aligned_returns = returns.iloc[max_lookback:]
        aligned_zones = zones[max_lookback:len(aligned_returns)+1]
        
        if len(aligned_returns) == 0 or len(aligned_zones) == 0:
            print("❌ Недостаточно данных для анализа")
            return {}
        
        # 1. ECONOMIC VALUE (40%)
        economic_value = self._calculate_economic_value(aligned_returns, aligned_zones)
        
        # 2. ZONE STABILITY (25%)
        zone_stability = self._calculate_zone_stability(aligned_zones)
        
        # 3. SIGNAL CONSISTENCY (20%)
        signal_consistency = self._calculate_signal_consistency(aligned_zones, aligned_returns)
        
        # 4. ADAPTABILITY SCORE (15%)
        adaptability_score = self._calculate_adaptability_score(params)
        
        # Композитный скор
        composite_score = (
            economic_value * 0.4 +
            zone_stability * 0.25 +
            signal_consistency * 0.2 +
            adaptability_score * 0.15
        )
        
        # Дополнительные метрики
        zone_distribution = self._calculate_zone_distribution(aligned_zones)
        performance_by_zone = self._calculate_performance_by_zone(aligned_returns, aligned_zones)
        
        results = {
            'timeframe': timeframe,
            'economic_value': economic_value,
            'zone_stability': zone_stability,
            'signal_consistency': signal_consistency,
            'adaptability_score': adaptability_score,
            'composite_score': composite_score,
            'zone_distribution': zone_distribution,
            'performance_by_zone': performance_by_zone,
            'total_periods': len(aligned_zones),
            'zone_changes': np.sum(np.diff(aligned_zones) != 0)
        }
        
        self._print_analysis_results(results)
        return results
    
    def _calculate_economic_value(self, returns: pd.Series, zones: np.ndarray) -> float:
        """Вычисляет Economic Value"""
        bull_returns = returns[zones == 1]
        bear_returns = returns[zones == -1]
        sideways_returns = returns[zones == 0]
        
        if len(bull_returns) == 0 or len(bear_returns) == 0:
            return 0.0
        
        return_spread = abs(bull_returns.mean() - bear_returns.mean())
        sideways_volatility = sideways_returns.std() if len(sideways_returns) > 0 else 1
        
        economic_value = return_spread / (1 + sideways_volatility)
        return max(economic_value, 0)
    
    def _calculate_zone_stability(self, zones: np.ndarray) -> float:
        """Вычисляет стабильность зон"""
        if len(zones) == 0:
            return 0.0
        
        zone_changes = np.sum(np.diff(zones) != 0)
        stability = 1 - (zone_changes / len(zones))
        return max(stability, 0)
    
    def _calculate_signal_consistency(self, zones: np.ndarray, returns: pd.Series) -> float:
        """Вычисляет согласованность сигналов"""
        if len(zones) == 0 or len(returns) == 0:
            return 0.0
        
        correct_signals = 0
        total_signals = 0
        
        for i in range(1, len(zones)):
            if zones[i] != 0:  # Не нейтральная зона
                price_direction = 1 if returns.iloc[i] > 0 else -1 if returns.iloc[i] < 0 else 0
                if zones[i] == price_direction:
                    correct_signals += 1
                total_signals += 1
        
        return correct_signals / total_signals if total_signals > 0 else 0.0
    
    def _calculate_adaptability_score(self, params: dict) -> float:
        """Вычисляет адаптивность системы"""
        score = 1.0
        
        # Проверяем соотношения параметров
        if params.get('fastMALength', 20) >= params.get('slowMALength', 50):
            score -= 0.3
        
        if params.get('macdFast', 12) >= params.get('macdSlow', 26):
            score -= 0.3
        
        # Проверяем веса
        total_weight = (params.get('trendWeightBase', 40) + 
                       params.get('momentumWeightBase', 30) + 
                       params.get('priceActionWeightBase', 30))
        
        if abs(total_weight - 100) > 5:  # Допуск 5%
            score -= 0.2
        
        return max(score, 0)
    
    def _calculate_zone_distribution(self, zones: np.ndarray) -> dict:
        """Вычисляет распределение зон"""
        total = len(zones)
        if total == 0:
            return {'bull': 0, 'bear': 0, 'sideways': 0}
        
        return {
            'bull': np.sum(zones == 1) / total,
            'bear': np.sum(zones == -1) / total,
            'sideways': np.sum(zones == 0) / total
        }
    
    def _calculate_performance_by_zone(self, returns: pd.Series, zones: np.ndarray) -> dict:
        """Вычисляет производительность по зонам"""
        bull_returns = returns[zones == 1]
        bear_returns = returns[zones == -1]
        sideways_returns = returns[zones == 0]
        
        return {
            'bull_mean': bull_returns.mean() if len(bull_returns) > 0 else 0,
            'bull_std': bull_returns.std() if len(bull_returns) > 0 else 0,
            'bear_mean': bear_returns.mean() if len(bear_returns) > 0 else 0,
            'bear_std': bear_returns.std() if len(bear_returns) > 0 else 0,
            'sideways_mean': sideways_returns.mean() if len(sideways_returns) > 0 else 0,
            'sideways_std': sideways_returns.std() if len(sideways_returns) > 0 else 0
        }
    
    def _print_analysis_results(self, results: dict):
        """Выводит результаты анализа"""
        print(f"\n📊 РЕЗУЛЬТАТЫ АНАЛИЗА:")
        print(f"   🎯 Economic Value: {results['economic_value']:.6f}")
        print(f"   🔄 Zone Stability: {results['zone_stability']:.3f}")
        print(f"   ✅ Signal Consistency: {results['signal_consistency']:.3f}")
        print(f"   🧠 Adaptability Score: {results['adaptability_score']:.3f}")
        print(f"   🏆 Composite Score: {results['composite_score']:.6f}")
        
        print(f"\n📈 РАСПРЕДЕЛЕНИЕ ЗОН:")
        dist = results['zone_distribution']
        print(f"   🟢 Bull: {dist['bull']:.1%}")
        print(f"   🔴 Bear: {dist['bear']:.1%}")
        print(f"   ⚪ Sideways: {dist['sideways']:.1%}")
        
        print(f"\n💰 ПРОИЗВОДИТЕЛЬНОСТЬ ПО ЗОНАМ:")
        perf = results['performance_by_zone']
        print(f"   🟢 Bull: {perf['bull_mean']:.4f} ± {perf['bull_std']:.4f}")
        print(f"   🔴 Bear: {perf['bear_mean']:.4f} ± {perf['bear_std']:.4f}")
        print(f"   ⚪ Sideways: {perf['sideways_mean']:.4f} ± {perf['sideways_std']:.4f}")
        
        print(f"\n📊 СТАТИСТИКА:")
        print(f"   📅 Всего периодов: {results['total_periods']}")
        print(f"   🔄 Смен зон: {results['zone_changes']}")
        print(f"   📊 Частота смен: {results['zone_changes']/results['total_periods']:.1%}")
        
        # Оценка качества
        self._evaluate_quality(results)
    
    def _evaluate_quality(self, results: dict):
        """Оценивает качество MZA"""
        print(f"\n🎯 ОЦЕНКА КАЧЕСТВА:")
        
        # Economic Value
        ev = results['economic_value']
        if ev > 0.01:
            ev_grade = "🟢 ОТЛИЧНО"
        elif ev > 0.005:
            ev_grade = "🟡 ХОРОШО"
        elif ev > 0.001:
            ev_grade = "🟠 УДОВЛЕТВОРИТЕЛЬНО"
        else:
            ev_grade = "🔴 ПЛОХО"
        print(f"   Economic Value: {ev_grade}")
        
        # Zone Stability
        zs = results['zone_stability']
        if zs > 0.8:
            zs_grade = "🟢 ОЧЕНЬ СТАБИЛЬНО"
        elif zs > 0.6:
            zs_grade = "🟡 СТАБИЛЬНО"
        elif zs > 0.4:
            zs_grade = "🟠 НЕСТАБИЛЬНО"
        else:
            zs_grade = "🔴 ОЧЕНЬ НЕСТАБИЛЬНО"
        print(f"   Zone Stability: {zs_grade}")
        
        # Signal Consistency
        sc = results['signal_consistency']
        if sc > 0.6:
            sc_grade = "🟢 ВЫСОКАЯ ТОЧНОСТЬ"
        elif sc > 0.5:
            sc_grade = "🟡 СРЕДНЯЯ ТОЧНОСТЬ"
        else:
            sc_grade = "🔴 НИЗКАЯ ТОЧНОСТЬ"
        print(f"   Signal Consistency: {sc_grade}")
        
        # Общая оценка
        composite = results['composite_score']
        if composite > 0.01:
            overall_grade = "🟢 ОТЛИЧНОЕ КАЧЕСТВО"
        elif composite > 0.005:
            overall_grade = "🟡 ХОРОШЕЕ КАЧЕСТВО"
        elif composite > 0.001:
            overall_grade = "🟠 УДОВЛЕТВОРИТЕЛЬНОЕ КАЧЕСТВО"
        else:
            overall_grade = "🔴 ПЛОХОЕ КАЧЕСТВО"
        print(f"\n🏆 ОБЩАЯ ОЦЕНКА: {overall_grade}")

print("✅ Класс MZAQualityAnalyzer создан")


✅ Класс MZAQualityAnalyzer создан


In [4]:
# Загружаем данные
print("📊 Загружаем данные BTC...")
btc_data = load_btc_data()

print("✅ Данные загружены:")
for tf, data in btc_data.items():
    print(f"   {tf}: {len(data)} записей")


📊 Загружаем данные BTC...
📊 ЗАГРУЗКА ДАННЫХ BTC ИЗ ФАЙЛОВ
✅ 15m: Загружены данные из df_btc_15m_complete.csv (61000 записей)
✅ 30m: Загружены данные из df_btc_30m_complete.csv (30000 записей)
✅ 1h: Загружены данные из df_btc_1h_complete.csv (15000 записей)
✅ 4h: Загружены данные из df_btc_4h_complete.csv (3000 записей)
✅ 1d: Загружены данные из df_btc_1d.csv (401 записей)
⚠️ 1d: Volume отсутствует, добавляем синтетический

📈 ИТОГО: 109,401 записей
📊 Таймфреймов: 5
✅ Данные загружены:
   15m: 61000 записей
   30m: 30000 записей
   1h: 15000 записей
   4h: 3000 записей
   1d: 401 записей


In [5]:
# Параметры вашего оптимизированного MZA для 15m
params_15m = {
    'adxLength': 16,
    'adxThreshold': 25,
    'fastMALength': 18,
    'slowMALength': 45,
    'rsiLength': 12,
    'stochKLength': 16,
    'macdFast': 10,
    'macdSlow': 24,
    'macdSignal': 8,
    'hhllRange': 22,
    'haDojiRange': 6,
    'candleRangeLength': 10,
    'bbLength': 18,
    'bbMultiplier': 2.2,
    'atrLength': 12,
    'kcLength': 18,
    'kcMultiplier': 1.8,
    'volumeMALength': 18,
    'trendWeightBase': 42,
    'momentumWeightBase': 32,
    'priceActionWeightBase': 26,
    'useSmoothing': True,
    'useHysteresis': True
}

print("✅ Параметры MZA для 15m загружены")
print(f"📊 Всего параметров: {len(params_15m)}")


✅ Параметры MZA для 15m загружены
📊 Всего параметров: 23


In [6]:
# Анализируем качество MZA на 15m
print("🔍 АНАЛИЗ КАЧЕСТВА MZA НА 15m")
print("=" * 50)

# Создаем классификатор
classifier_15m = AccurateMZAClassifier(params_15m)

# Получаем данные для 15m
data_15m = btc_data['15m']

# Получаем предсказания зон
print("🧠 Вычисляем зоны для 15m...")
zones_15m = classifier_15m.predict(data_15m)

print(f"✅ Зоны вычислены: {len(zones_15m)} периодов")
print(f"📊 Распределение зон:")
print(f"   🟢 Bull: {np.sum(zones_15m == 1)} ({np.sum(zones_15m == 1)/len(zones_15m):.1%})")
print(f"   🔴 Bear: {np.sum(zones_15m == -1)} ({np.sum(zones_15m == -1)/len(zones_15m):.1%})")
print(f"   ⚪ Sideways: {np.sum(zones_15m == 0)} ({np.sum(zones_15m == 0)/len(zones_15m):.1%})")


🔍 АНАЛИЗ КАЧЕСТВА MZA НА 15m
🧠 Вычисляем зоны для 15m...
✅ Зоны вычислены: 61000 периодов
📊 Распределение зон:
   🟢 Bull: 2209 (3.6%)
   🔴 Bear: 2280 (3.7%)
   ⚪ Sideways: 56511 (92.6%)


In [8]:
# Запускаем детальный анализ качества
analyzer = MZAQualityAnalyzer()
results_15m = analyzer.analyze_mza_quality(data_15m, zones_15m, params_15m, "15m")


🔍 АНАЛИЗ КАЧЕСТВА MZA ДЛЯ 15m


IndexError: Boolean index has wrong length: 60910 instead of 60954

In [ ]:
# Параметры для 1h (из вашей оптимизации)
params_1h = {
    'adxLength': 15,
    'adxThreshold': 22,
    'fastMALength': 19,
    'slowMALength': 47,
    'rsiLength': 13,
    'stochKLength': 15,
    'macdFast': 11,
    'macdSlow': 25,
    'macdSignal': 9,
    'hhllRange': 21,
    'haDojiRange': 7,
    'candleRangeLength': 11,
    'bbLength': 19,
    'bbMultiplier': 2.1,
    'atrLength': 13,
    'kcLength': 19,
    'kcMultiplier': 1.7,
    'volumeMALength': 19,
    'trendWeightBase': 41,
    'momentumWeightBase': 33,
    'priceActionWeightBase': 26,
    'useSmoothing': True,
    'useHysteresis': True
}

print("✅ Параметры MZA для 1h загружены")


In [ ]:
# Анализируем качество MZA на 1h
print("\n🔍 АНАЛИЗ КАЧЕСТВА MZA НА 1h")
print("=" * 50)

# Создаем классификатор для 1h
classifier_1h = AccurateMZAClassifier(params_1h)

# Получаем данные для 1h
data_1h = btc_data['1h']

# Получаем предсказания зон
print("🧠 Вычисляем зоны для 1h...")
zones_1h = classifier_1h.predict(data_1h)

print(f"✅ Зоны вычислены: {len(zones_1h)} периодов")
print(f"📊 Распределение зон:")
print(f"   🟢 Bull: {np.sum(zones_1h == 1)} ({np.sum(zones_1h == 1)/len(zones_1h):.1%})")
print(f"   🔴 Bear: {np.sum(zones_1h == -1)} ({np.sum(zones_1h == -1)/len(zones_1h):.1%})")
print(f"   ⚪ Sideways: {np.sum(zones_1h == 0)} ({np.sum(zones_1h == 0)/len(zones_1h):.1%})")


In [ ]:
# Запускаем детальный анализ качества для 1h
results_1h = analyzer.analyze_mza_quality(data_1h, zones_1h, params_1h, "1h")


In [ ]:
# Сравнение результатов по таймфреймам
print("\n📊 СРАВНЕНИЕ ПО ТАЙМФРЕЙМАМ")
print("=" * 60)

# Создаем DataFrame для сравнения
comparison_data = [
    {
        'timeframe': '15m',
        'economic_value': results_15m['economic_value'],
        'zone_stability': results_15m['zone_stability'],
        'signal_consistency': results_15m['signal_consistency'],
        'composite_score': results_15m['composite_score']
    },
    {
        'timeframe': '1h',
        'economic_value': results_1h['economic_value'],
        'zone_stability': results_1h['zone_stability'],
        'signal_consistency': results_1h['signal_consistency'],
        'composite_score': results_1h['composite_score']
    }
]

df_comparison = pd.DataFrame(comparison_data)

print(f"{'Таймфрейм':<8} {'Economic Value':<15} {'Zone Stability':<15} {'Signal Cons':<12} {'Composite':<12}")
print("-" * 70)

for _, row in df_comparison.iterrows():
    print(f"{row['timeframe']:<8} {row['economic_value']:<15.6f} {row['zone_stability']:<15.3f} "
          f"{row['signal_consistency']:<12.3f} {row['composite_score']:<12.6f}")

# Находим лучший таймфрейм
best_idx = df_comparison['composite_score'].idxmax()
best_timeframe = df_comparison.loc[best_idx, 'timeframe']
best_score = df_comparison.loc[best_idx, 'composite_score']

print(f"\n🏆 ЛУЧШИЙ ТАЙМФРЕЙМ: {best_timeframe} (Composite Score: {best_score:.6f})")


In [ ]:
# Рекомендации по улучшению MZA
print(f"\n🎯 РЕКОМЕНДАЦИИ ПО УЛУЧШЕНИЮ MZA:")
print("=" * 50)

# Анализируем проблемы
print(f"\n🔍 АНАЛИЗ ПРОБЛЕМ:")

for result in [results_15m, results_1h]:
    tf = result['timeframe']
    ev = result['economic_value']
    zs = result['zone_stability']
    sc = result['signal_consistency']
    
    print(f"\n📊 {tf}:")
    
    if ev < 0.001:
        print(f"   ❌ Economic Value слишком низкий ({ev:.6f})")
        print(f"      💡 Рекомендация: Увеличьте adxThreshold или используйте другой таймфрейм")
    elif ev > 0.005:
        print(f"   ✅ Economic Value отличный ({ev:.6f})")
    
    if zs < 0.4:
        print(f"   ❌ Зоны нестабильны ({zs:.3f})")
        print(f"      💡 Рекомендация: Включите useSmoothing=True и useHysteresis=True")
    elif zs > 0.6:
        print(f"   ✅ Зоны стабильны ({zs:.3f})")
    
    if sc < 0.5:
        print(f"   ❌ Низкая точность сигналов ({sc:.3f})")
        print(f"      💡 Рекомендация: Переоптимизируйте параметры или используйте другой таймфрейм")
    elif sc > 0.6:
        print(f"   ✅ Высокая точность сигналов ({sc:.3f})")
    
    if ev > 0.005 and zs > 0.6 and sc > 0.5:
        print(f"   ✅ Качество хорошее!")

# Общие рекомендации
print(f"\n🚀 ОБЩИЕ РЕКОМЕНДАЦИИ:")
print("   1. 📊 Используйте лучший таймфрейм для торговли")
print("   2. 🔄 Регулярно переоптимизируйте параметры (раз в месяц)")
print("   3. 📈 Не полагайтесь только на MZA - используйте дополнительные индикаторы")
print("   4. ⚠️ Тестируйте на разных активах и рыночных условиях")
print("   5. 🎯 Фокусируйтесь на Economic Value > 0.005 для хорошего качества")
print("   6. 🔍 Анализируйте качество зон перед использованием в торговле")


In [ ]:
# Создаем визуальный анализ
print(f"\n📊 СОЗДАЕМ ВИЗУАЛЬНЫЙ АНАЛИЗ...")

# Подготавливаем данные для графиков
timeframes = ['15m', '1h']
economic_values = [results_15m['economic_value'], results_1h['economic_value']]
zone_stabilities = [results_15m['zone_stability'], results_1h['zone_stability']]
signal_consistencies = [results_15m['signal_consistency'], results_1h['signal_consistency']]
composite_scores = [results_15m['composite_score'], results_1h['composite_score']]

# Создаем графики
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Анализ качества MZA по таймфреймам', fontsize=16)

# Economic Value
axes[0, 0].bar(timeframes, economic_values, color=['green', 'blue'])
axes[0, 0].set_title('Economic Value')
axes[0, 0].set_ylabel('Economic Value')
axes[0, 0].axhline(y=0.005, color='red', linestyle='--', label='Хорошее качество')
axes[0, 0].legend()

# Zone Stability
axes[0, 1].bar(timeframes, zone_stabilities, color=['green', 'blue'])
axes[0, 1].set_title('Zone Stability')
axes[0, 1].set_ylabel('Zone Stability')
axes[0, 1].axhline(y=0.6, color='red', linestyle='--', label='Стабильные зоны')
axes[0, 1].legend()

# Signal Consistency
axes[1, 0].bar(timeframes, signal_consistencies, color=['green', 'blue'])
axes[1, 0].set_title('Signal Consistency')
axes[1, 0].set_ylabel('Signal Consistency')
axes[1, 0].axhline(y=0.5, color='red', linestyle='--', label='Средняя точность')
axes[1, 0].legend()

# Composite Score
axes[1, 1].bar(timeframes, composite_scores, color=['green', 'blue'])
axes[1, 1].set_title('Composite Score')
axes[1, 1].set_ylabel('Composite Score')
axes[1, 1].axhline(y=0.005, color='red', linestyle='--', label='Хорошее качество')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("✅ График создан!")


## 🎯 ЗАКЛЮЧЕНИЕ

### 📊 **КЛЮЧЕВЫЕ ВЫВОДЫ:**

1. **🎯 Economic Value** - главная метрика качества MZA
   - Показывает, насколько хорошо система разделяет бычьи и медвежьи зоны
   - Ваш результат 0.592989 на 15m - это **ОТЛИЧНЫЙ** результат!

2. **🔄 Zone Stability** - стабильность зон
   - Показывает, как часто система меняет свое мнение
   - Высокая стабильность = меньше ложных сигналов

3. **✅ Signal Consistency** - точность сигналов
   - Показывает, насколько часто сигналы совпадают с движением цены
   - Важно для практического использования

4. **🧠 Adaptability Score** - разумность параметров
   - Проверяет корректность настроек системы

### 🚀 **РЕКОМЕНДАЦИИ:**

1. **Используйте лучший таймфрейм** - тот, где Composite Score максимальный
2. **Регулярно переоптимизируйте** - рынок меняется, параметры тоже должны
3. **Не полагайтесь только на MZA** - используйте дополнительные индикаторы
4. **Тестируйте на разных активах** - BTC может отличаться от других криптовалют
5. **Фокусируйтесь на Economic Value > 0.005** для хорошего качества

### ⚠️ **ВАЖНО ПОМНИТЬ:**

- MZA не предсказывает будущее, а классифицирует текущее состояние
- Он должен показывать зоны, а не точные точки входа/выхода
- Качество определяется математически, а не визуально
- Ваш Economic Value 0.592989 говорит о том, что система работает отлично!
